In [ ]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/content/qa_pairs.csv')

# Inspect data
print(df.head())

# Check for missing values
print(df.isnull().sum())

# Drop rows with missing values if any
df = df.dropna()

# Convert to JSON-like format for instruction tuning
df['data'] = df.apply(lambda row: {'instruction': 'Answer the following question based on your knowledge.',
                                   'question': row['Question'],
                                   'answer': row['Answer']}, axis=1)

# Save to a new JSON file
df['data'].to_json('formatted_dataset.json', orient='records')

# Split data into train, validate, and test sets
train, validate, test = np.split(df.sample(frac=1, random_state=42),
                                 [int(.8*len(df)), int(.9*len(df))])

                                            Question  \
0  What approach did the political parties in Pak...   
1  What is the US government's stance towards Pak...   
2  What is the root cause of the Muslims' resentm...   
3  What was the purpose of the Charter of Democracy?   
4  How did Nawaz Sharif react to a few members of...   

                                              Answer  
0  They aimed to bring peace through dialogue rat...  
1  The US government has decried the peace accord...  
2  The root causes are past and present discrimin...  
3  Nawaz Sharif and late Benazir Bhutto were goin...  
4  Nawaz Sharif thought for a long time and then ...  
Question    0
Answer      0
dtype: int64


In [ ]:
train

,Question,Answer,data
291,What is the Deoband seminary's view on terrori...,The prestigious seminary at Deoband has ruled ...,{'instruction': 'Answer the following question...
3056,What were the two instances where the Pakistan...,The Pakistani army was tested twice within a f...,{'instruction': 'Answer the following question...
2093,What are some reasons for corruption in the po...,Low pay and insufficient compensation packages...,{'instruction': 'Answer the following question...
432,What are the potential consequences of an Isra...,The fallout could be radioactive chaos and the...,{'instruction': 'Answer the following question...
479,What is the author's concern about the loss of...,The author is concerned that the loss of the s...,{'instruction': 'Answer the following question...
...,...,...,...
2220,What was the trauma of the Partition?,The refugees on both sides received a poor rec...,{'instruction': 'Answer the following question...
2979,What did the Prime Minister recently commit to...,The PM recently committed to fighting corrupti...,{'instruction': 'Answer the following question...
2105,What is the significance of the Sachar Report ...,The Sachar Report is a high powered report ini...,{'instruction': 'Answer the following question...
133,What is the proposed solution to dealing with ...,Catch the speculators and deal with them as th...,{'instruction': 'Answer the following question...


In [ ]:
# Function to save checkpoints
def save_checkpoint(model, optimizer, step, path="checkpoints"):
    if not os.path.exists(path):
        os.makedirs(path)
    checkpoint_path = os.path.join(path, f"checkpoint_{step}.pt")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved to {checkpoint_path}")

In [ ]:
def encode_data(tokenizer, questions, answers, max_length=128):
    # Check if the tokenizer has a padding token, set if not
    if tokenizer.pad_token is None:
        print("No padding token found. Using `eos_token` as `pad_token`.")
        tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

    return tokenizer(questions, answers, truncation=True, padding='max_length', max_length=max_length)

#**GPT2**

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
import torch
import torch.optim as optim

In [ ]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to('cuda')  # Ensure model is on GPU if available

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Check and set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
# Function to tokenize data
def encode_data(tokenizer, questions, answers):
    return tokenizer(questions, answers, truncation=True, padding='max_length', max_length=128)

In [ ]:
# Prepare datasets
train_encodings = encode_data(tokenizer, train['Question'].tolist(), train['Answer'].tolist())
train_dataset = QADataset(train_encodings)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Training loop
model.train()
step = 0
for epoch in range(3):  # number of epochs
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = input_ids
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        # print(f"Epoch {epoch}, Loss {loss.item()}")

        if (step + 1) % 1131 == 0:
            save_checkpoint(model, optimizer, step+1)

        step += 1
        print(f"Step {step}, Loss {loss.item()}")


Step 1, Loss 1.0972297191619873
Step 2, Loss 1.3311423063278198
Step 3, Loss 1.0650053024291992
Step 4, Loss 0.9423151612281799
Step 5, Loss 0.9773575663566589
Step 6, Loss 1.4125479459762573
Step 7, Loss 1.2452579736709595
Step 8, Loss 1.0443758964538574
Step 9, Loss 1.2652199268341064
Step 10, Loss 1.5285065174102783
Step 11, Loss 1.5010120868682861
Step 12, Loss 1.2695190906524658
Step 13, Loss 1.4262065887451172
Step 14, Loss 1.19707453250885
Step 15, Loss 0.859687328338623
Step 16, Loss 1.2455826997756958
Step 17, Loss 0.6871766448020935
Step 18, Loss 1.1261796951293945
Step 19, Loss 0.8600078821182251
Step 20, Loss 0.936225950717926
Step 21, Loss 1.1231290102005005
Step 22, Loss 0.9794762134552002
Step 23, Loss 1.319831371307373
Step 24, Loss 1.2341545820236206
Step 25, Loss 1.4068626165390015
Step 26, Loss 0.9944783449172974
Step 27, Loss 1.0996568202972412
Step 28, Loss 1.2179951667785645
Step 29, Loss 1.5271323919296265
Step 30, Loss 1.171280026435852
Step 31, Loss 1.147509932